In [3]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType, DoubleType
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import io


# INICIAR SPARK SESSION

spark = SparkSession.builder \
    .appName("CryptoVolatilityAnalysis") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

df_raw = spark.read.csv("crypto_volatility_dec2025_FINAL_2025-12-09.csv", header=True, inferSchema=True)

# TRATAMENTO DOS DADOS

df = df_raw \
    .withColumn("timestamp", to_timestamp("timestamp")) \
    .withColumn("close", col("close").cast(DoubleType())) \
    .withColumn("volume", col("volume").cast("double")) \
    .orderBy("timestamp")

# Remover linhas duplicadas ou completamente nulas (se houver)
df = df.dropDuplicates()

print(f"Total de linhas: {df.count():,}")
print("Amostra:")
df.show(10)

# ESTATÍSTICAS BÁSICAS POR ATIVO

stats = df.groupBy("asset", "symbol") \
    .agg(
        count("*").alias("entries"),
        avg("close").alias("avg_price"),
        stddev("close").alias("std_price"),
        min("close").alias("min_price"),
        max("close").alias("max_price"),
        percentile_approx("close", 0.25).alias("q25"),
        percentile_approx("close", 0.50).alias("q50"),
        percentile_approx("close", 0.75).alias("q75"),
        avg("volume").alias("avg_volume")
    ) \
    .orderBy(desc("entries"))

print("\nEstatísticas por ativo:")
stats.show(20, truncate=False)

# Converter para Pandas para facilitar gráficos
stats_pd = stats.toPandas()

# 6. VOLATILIDADE (desvio-padrão dos retornos %)

from pyspark.sql.window import Window

window_spec = Window.partitionBy("symbol").orderBy("timestamp")

df_with_return = df \
    .withColumn("prev_close", lag("close").over(window_spec)) \
    .withColumn("return", when(col("prev_close").isNotNull(),
                               (col("close") - col("prev_close")) / col("prev_close") * 100)
                .otherwise(None))

volatility = df_with_return \
    .groupBy("symbol") \
    .agg(
        stddev("return").alias("volatility_pct")
    ) \
    .orderBy(desc("volatility_pct"))

print("\nRanking de volatilidade (minute-level returns):")
volatility.show()

volatility_pd = volatility.toPandas()

# 7. GRÁFICOS DE PREÇO POR ATIVO

symbols = df.select("symbol").distinct().toPandas()["symbol"].tolist()

for sym in symbols:
    print(f"\nGerando gráfico para {sym}...")
    df_sym = df.filter(col("symbol") == sym).orderBy("timestamp").toPandas()

    # Amostrar para no máximo 500 pontos (Plotly fica lento com muitos pontos)
    if len(df_sym) > 500:
        df_sym = df_sym.iloc[::len(df_sym)//500].copy()

    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(f"Preço - {sym}", "Volume"),
        shared_xaxes=True,
        vertical_spacing=0.1,
        row_heights=[0.7, 0.3]
    )

    fig.add_trace(go.Scatter(x=df_sym['timestamp'], y=df_sym['close'],
                             mode='lines', name='Close Price',
                             line=dict(width=1.5)), row=1, col=1)

    fig.add_trace(go.Bar(x=df_sym['timestamp'], y=df_sym['volume'],
                         name='Volume', marker_color='lightgray'), row=2, col=1)

    fig.update_layout(height=600, title_text=f"{sym} - Dezembro 2025")
    fig.update_xaxes(title_text="Data/Hora", row=2, col=1)
    fig.update_yaxes(title_text="Preço (USD)", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.show()

# 8. MATRIZ DE CORRELAÇÃO ENTRE ATIVOS

# Pivot para ter uma coluna por ativo (close price)
pivot = df.select("timestamp", "symbol", "close") \
    .groupBy("timestamp") \
    .pivot("symbol") \
    .agg(first("close")) \
    .orderBy("timestamp") \
    .toPandas()

pivot = pivot.set_index("timestamp")

# Preencher valores faltantes com forward fill (muito comum em dados de minuto)
pivot_ffill = pivot.ffill().dropna()  # remove linhas que ainda têm NaN no início

corr_matrix = pivot_ffill.corr()

print("\nMatriz de Correlação (Close Prices):")
print(corr_matrix.round(2))

# Gráfico de calor da correlação
fig_corr = px.imshow(corr_matrix,
                     text_auto=True,
                     aspect="auto",
                     color_continuous_scale="RdBu",
                     title="Matriz de Correlação entre Criptomoedas (Dez/2025)")
fig_corr.update_layout(height=700)
fig_corr.show()

# 9. RANKING GERAL (VOLATILIDADE + VOLUME MÉDIO)

ranking = stats_pd.merge(volatility_pd, on="symbol") \
    .sort_values("volatility_pct", ascending=False) \
    [["symbol", "entries", "avg_price", "volatility_pct", "avg_volume"]]

ranking["volatility_pct"] = ranking["volatility_pct"].round(3)
ranking["avg_volume"] = ranking["avg_volume"].round(0).astype(int)

print("\nRanking Final - Volatilidade vs Volume Médio:")
print(ranking)

# Gráfico combinado
fig_rank = px.scatter(ranking,
                      x="volatility_pct",
                      y="avg_volume",
                      size="entries",
                      color="symbol",
                      text="symbol",
                      title="Volatilidade x Volume Médio (tamanho = nº de registros)",
                      labels={"volatility_pct": "Volatilidade (% retorno minuto)",
                              "avg_volume": "Volume Médio"})
fig_rank.update_traces(textposition="top center")
fig_rank.show()

print("\nAnálise concluída! Todos os gráficos foram exibidos acima.")

Total de linhas: 65,201
Amostra:
+-------------------+-------------+-------------+-------------+-------------+-------------+-------+-------+
|          timestamp|         open|         high|          low|        close|       volume|  asset| symbol|
+-------------------+-------------+-------------+-------------+-------------+-------------+-------+-------+
|2025-12-03 09:18:00| 92923.515625| 92923.515625| 92923.515625| 92923.515625|          0.0|Bitcoin|BTC-USD|
|2025-12-03 13:11:00|93033.7421875|93033.7421875|93033.7421875|93033.7421875|          0.0|Bitcoin|BTC-USD|
|2025-12-03 17:16:00| 92658.015625| 92658.015625| 92658.015625| 92658.015625|          0.0|Bitcoin|BTC-USD|
|2025-12-03 17:42:00| 92854.453125| 92854.453125| 92854.453125| 92854.453125|2.212716544E9|Bitcoin|BTC-USD|
|2025-12-03 18:31:00|     93158.75|     93158.75|     93158.75|     93158.75|          0.0|Bitcoin|BTC-USD|
|2025-12-03 20:11:00|93056.0703125|93056.0703125|93056.0703125|93056.0703125|          0.0|Bitcoin|BTC-


Gerando gráfico para BONK-USD...



Gerando gráfico para WIF-USD...



Gerando gráfico para XRP-USD...



Gerando gráfico para PEPE24478-USD...


...


Gerando gráfico para NEAR-USD...



Gerando gráfico para RENDER-USD...



Matriz de Correlação (Close Prices):
               BONK-USD  BTC-USD  DOGE-USD  ETH-USD  NEAR-USD  PEPE24478-USD  \
BONK-USD           1.00     0.84      0.88     0.61      0.90           0.84   
BTC-USD            0.84     1.00      0.95     0.71      0.94           0.81   
DOGE-USD           0.88     0.95      1.00     0.65      0.94           0.83   
ETH-USD            0.61     0.71      0.65     1.00      0.74           0.82   
NEAR-USD           0.90     0.94      0.94     0.74      1.00           0.88   
PEPE24478-USD      0.84     0.81      0.83     0.82      0.88           1.00   
RENDER-USD         0.86     0.93      0.97     0.62      0.93           0.78   
SHIB-USD           0.85     0.83      0.90     0.60      0.84           0.77   
SOL-USD            0.85     0.95      0.98     0.66      0.94           0.80   
WIF-USD            0.91     0.85      0.87     0.63      0.91           0.87   
XRP-USD            0.88     0.93      0.94     0.55      0.91           0.75   

 


Ranking Final - Volatilidade vs Volume Médio:
           symbol  entries     avg_price  volatility_pct  avg_volume
8         WIF-USD     3294      0.388795           0.200      108454
9      RENDER-USD     3054      1.676824           0.164       26125
6        BONK-USD     3778      0.000010           0.158       52351
10       SHIB-USD     3003      0.000009           0.133       92862
7         SOL-USD     3450    137.667881           0.125     2602225
4   PEPE24478-USD     7657      0.000005           0.114      118469
5        NEAR-USD     5565      1.759680           0.101       67019
2         ETH-USD     8784   3099.368105           0.087    65911670
3         BTC-USD     8687  91069.085328           0.059    83935326
0        DOGE-USD     9046      0.143686           0.055      329736
1         XRP-USD     8883      2.088017           0.049      697375



Análise concluída! Todos os gráficos foram exibidos acima.
